In [1]:
#Import the right libraries
import pandas as pd
import dgl
import numpy as np
import networkx as nx
import torch

Using backend: pytorch


In [2]:
#Import the data
company_data = pd.read_csv('Company.dat', sep="|")
investor_data = pd.read_csv('Investor.dat', sep="|")
relation_data = pd.read_csv('InvestorInvestmentRelation.dat', sep="|")

/home/ramon/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/ramon/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (97) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Get the unique ids for investors and companies in the relation data
investor_ids_relation = relation_data['InvestorID'].unique()
company_ids_relation = relation_data['CompanyID'].unique()

print('We have', len(investor_ids_relation), 'unique investor IDs in the relation data')
print('We have', len(company_ids_relation), 'unique company IDs in the relation data')

We have 7367 unique investor IDs in the relation data
We have 186495 unique company IDs in the relation data


In [4]:
#Get the unique ids for investors and companies in their own datafile
investor_ids = investor_data['InvestorID'].unique()
company_ids = company_data['CompanyID'].unique()

#Check whether the relation ids occur in the investor_ids or the company_ids
common_ids_investor = set(investor_ids_relation) & set(investor_ids)
common_ids_investor2 = set(investor_ids_relation) & set(company_ids)

#Add the two sets to get the unique investor ids
common_investor_ids = common_ids_investor | common_ids_investor2

#Check if they occur in both
common_common = common_ids_investor & common_ids_investor2

common_ids_company = set(company_ids_relation) & set(investor_ids)
common_ids_company2 = set(company_ids_relation) & set(company_ids)

#Add the two sets and take the set again to get the unique values
common_company_ids = common_ids_company | common_ids_company2

common_common2 = common_ids_company & common_ids_company2


print('The amount of unique investor ids in the relation dataset:', len(investor_ids_relation))
print('The amount of unique investor ids in the relation dataset that can be found in the investor dataset:',
      len(common_ids_investor))
print('The amount of unique investor ids in the relation dataset that can be found in the company dataset:',
      len(common_ids_investor2))
print('The amount of unique investor ids in the relation dataset that can be found in the investor AND the company dataset:',
      len(common_common))
print('Therefore we have', len(common_investor_ids), 'unique investor ids')
print(len(investor_ids_relation) - len(common_ids_investor) - len(common_ids_investor2), 'investor ids do not have metadata')

print('\n')

print('The amount of unique company ids in the relation dataset:', len(company_ids_relation))
print('The amount of unique company ids in the relation dataset that can be found in the investor dataset:',
      len(common_ids_company))
print('The amount of unique company ids in the relation dataset that can be found in the company dataset:',
      len(common_ids_company2))
print('The amount of unique company ids in the relation dataset that can be found in the investor AND the company dataset:',
      len(common_common2))
print('Therefore we have', len(common_company_ids), 'unique company ids')
print(len(company_ids_relation) - len(common_ids_company) - len(common_ids_company2), 'company ids do not have metadata')

The amount of unique investor ids in the relation dataset: 7367
The amount of unique investor ids in the relation dataset that can be found in the investor dataset: 7320
The amount of unique investor ids in the relation dataset that can be found in the company dataset: 29
The amount of unique investor ids in the relation dataset that can be found in the investor AND the company dataset: 29
Therefore we have 7320 unique investor ids
18 investor ids do not have metadata


The amount of unique company ids in the relation dataset: 186495
The amount of unique company ids in the relation dataset that can be found in the investor dataset: 1021
The amount of unique company ids in the relation dataset that can be found in the company dataset: 9779
The amount of unique company ids in the relation dataset that can be found in the investor AND the company dataset: 75
Therefore we have 10725 unique company ids
175695 company ids do not have metadata


PROBLEM: not all the id's in the investor_ids/company_ids from the relation data are in the pure company and pure investor dataset. This means that for some relations we only have a investor id or company id but no additional information.

The graph does not take strings as input so it has to be a number. What are the options?
- We could extract the investor ID's and company ID's and transform these intoo the indices from the pure company and pure investor dataset. However, this is not possible because we do not have all the IDs

- CURRENT CHOICE: We could skip them and only use the nodes where we have metadata from

In [5]:
#Only get the investors and companies where we have metadata from
processed_relation_data = relation_data.query('(InvestorID in @common_investor_ids) & (CompanyID in @common_company_ids)').reset_index(drop=True)
print(len(processed_relation_data.InvestorID.unique()))
print(len(processed_relation_data.CompanyID.unique()))

7320
10722


In [6]:
#Get the relevant investor ids and company ids from the original dataset and reset the index for the input of dgl.
processed_investor_data = investor_data.query('(InvestorID in @common_investor_ids) | (InvestorID in @common_company_ids)').reset_index(drop=True)
processed_company_data = company_data.query('(CompanyID in @common_company_ids)').reset_index(drop=True)

print(len(processed_investor_data))
print(len(processed_company_data))

tuples_relations = list(zip(processed_relation_data.InvestorID, processed_relation_data.CompanyID))

7883
9779


In [7]:
print(len(processed_company_data))
print(len(processed_investor_data))
print(len(processed_relation_data['CompanyID'].unique()))
print(len(processed_relation_data['InvestorID'].unique()))

9779
7883
10722
7320


In [8]:
print('From the', len(investor_data), 'investor data nodes we can only use', len(processed_investor_data), 'nodes.')
print('From the', len(company_data), 'company data nodes we can only use', len(processed_company_data), 'nodes.')
print('From the', len(relation_data), 'relation edges we can only use', len(processed_relation_data), 'edges.')

From the 18377 investor data nodes we can only use 7883 nodes.
From the 11377 company data nodes we can only use 9779 nodes.
From the 519123 relation edges we can only use 55189 edges.


# Save the files

In [9]:
#Create new processed csv files
# processed_investor_data.to_csv('processed_investor_data.csv', index=False)
# processed_company_data.to_csv('processed_company_data.csv', index=False)
# processed_relation_data.to_csv('processed_relation_data.csv', index=False)